# measure the average minimum distance between two lines, or the zonal stats around each line's buffer

In [1]:
from geopandas import read_file, GeoSeries, GeoDataFrame
from shapely.geometry import Point

import numpy as np
from rioxarray import open_rasterio
from rasterstats import zonal_stats

from pathlib import Path
from xarray import DataArray
from pandas import DataFrame
from shapely.geometry import LineString, Polygon
from shapely.validation import make_valid

from src.rules2weights import write_compressed

# distances
Distance between two paths
## Distance between all touched True and All Touched False

In [2]:
def compute_line_distances(line1: LineString, line2: LineString) -> list:
    return [Point(p).distance(line2) for p in line1.coords]

In [3]:
resolutions = [5, 10, 25, 50, 100]

least_cost_path_5_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_false.json")
least_cost_path_5_false = least_cost_path_5_false.to_crs(25832)

paths_false = [least_cost_path_5_false,
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_false.gpkg"),
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_false.gpkg"),
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_false.gpkg"),
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_false.gpkg")]
paths_true = [read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_true.gpkg")]

path_lengths_false = [x.geometry.values[0].length for x in paths_false]
path_lengths_true = [x.geometry.values[0].length for x in paths_true]

distances = [compute_line_distances(t.geometry.values[0], f.geometry.values[0]) for f, t in
             zip(paths_false, paths_true)]
mean_distances = [np.mean(x) for x in distances]
max_distances = [np.max(x) for x in distances]

DataFrame({'Path lengths all touched false': path_lengths_false,
           'Path lengths all touched true': path_lengths_true,
           'mean distances': mean_distances,
           'max distances': max_distances},
          index=resolutions)

,Path lengths all touched false,Path lengths all touched true,mean distances,max distances
5,76136.340836,78002.006658,146.446922,1333.014707
10,75430.100870,77936.568748,313.612446,1847.836573
25,75422.854133,78422.854133,337.306328,1909.188309
50,76135.021342,70619.945583,793.807567,3748.332963
100,76283.807115,74120.732103,1827.133061,6574.191966


## distance - between path of 5m path, and paths from 5m down sampled
and compared the path for the down sampled variant to the original one of the resolution

In [4]:
# original path 5 m resolution
least_cost_path_5_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_false.json")
least_cost_path_5_false = least_cost_path_5_false.to_crs(25832)

down_sampled_resolutions = [10, 25, 50, 100]

# paths from bi-linear down sampled  raster
down_sampled = [read_file("..\\results\\least_cost_paths\\Downsampled\\least_cost_path_test_points_res_5_al_false_downsampled.gpkg"),
               read_file("..\\results\\least_cost_paths\\Downsampled\\least_cost_path_test_points_res_5_al_false_downsampled_25.gpkg"),
               read_file("..\\results\\least_cost_paths\\Downsampled\\least_cost_path_test_points_res_5_al_false_downsampled_50.gpkg"),
               read_file("..\\results\\least_cost_paths\\Downsampled\\least_cost_path_test_points_res_5_al_false_downsampled_100.gpkg"),
               ]
path_lengths_downsampled = [x.geometry.values[0].length for x in down_sampled]

# original paths, from lower resolution raster, all touched false
original_false = [read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_false.gpkg"),
                  read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_false.gpkg"),
                  read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_false.gpkg"),
                  read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_false.gpkg"),
                 ]

# original paths, from lower resolution raster, all touched false
original_true = [read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_true.gpkg"),
                  read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_true.gpkg"),
                  read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_true.gpkg"),
                  read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_true.gpkg"),
                 ]

distances_compared_to_5m = [compute_line_distances(d.geometry.values[0], least_cost_path_5_false.geometry.values[0]) for d in down_sampled]
mean_distances_5m = [np.mean(x) for x in distances_compared_to_5m]

distances_compared_to_original_false = [compute_line_distances(d.geometry.values[0], o.geometry.values[0]) for d, o in zip(down_sampled, original_false)]
mean_distances_original_false = [np.mean(x) for x in distances_compared_to_original_false]

distances_compared_to_original_true = [compute_line_distances(d.geometry.values[0], o.geometry.values[0]) for d, o in zip(down_sampled, original_true)]
mean_distances_original_true = [np.mean(x) for x in distances_compared_to_original_true]


DataFrame({'Path lengths down sampled': path_lengths_downsampled,
           'mean distances to 5 m path': mean_distances_5m,
           'relative costs': [x['total cost'][0]* res for x, res in zip(down_sampled, down_sampled_resolutions)],
           'mean distances to original resolution, false': mean_distances_original_false,
           'mean distances to original resolution, true': mean_distances_original_true},
          index=down_sampled_resolutions)

,Path lengths down sampled,mean distances to 5 m path,relative costs,"mean distances to original resolution, false","mean distances to original resolution, true"
10,75980.619569,59.342459,94136.15,219.349393,143.633075
25,70205.265747,385.787115,91844.35,558.124307,432.774864
50,69217.864055,730.819050,92091.25,693.422086,255.695791
100,66667.869159,1681.305176,91701.50,1605.632877,400.546818


## distance of paths from mixed all touched

In [8]:
# original path 5 m resolution
least_cost_path_5_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_false.json")
least_cost_path_5_false = least_cost_path_5_false.to_crs(25832)

# original path, from lower resolution raster, all touched false
least_cost_path_10_false = read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_false.gpkg")

# original path, from lower resolution raster, all touched false
least_cost_path_10_true = read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_true.gpkg")

r_false = [1, 2, 4, 8, 16, 32, 64, 128]

# paths from mixed all touched
mixed = [read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_1_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_2_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_4_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_8_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_16_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_32_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_64_1.gpkg"),
         read_file("..\\results\\least_cost_paths\\Mixed_10_m\\least_cost_path_test_points_res_10_al_mixed_128_1.gpkg"),
        ]
path_lengths_mixed = [x.geometry.values[0].length for x in mixed]

distances_compared_to_5m = [compute_line_distances(m.geometry.values[0], least_cost_path_5_false.geometry.values[0]) for m in mixed]
mean_distances_5m = [np.mean(x) for x in distances_compared_to_5m]

distances_compared_to_10m_false = [compute_line_distances(m.geometry.values[0], least_cost_path_10_false.geometry.values[0]) for m in mixed]
mean_distances_10m_false = [np.mean(x) for x in distances_compared_to_10m_false]

distances_compared_to_10m_true = [compute_line_distances(m.geometry.values[0], least_cost_path_10_true.geometry.values[0]) for m in mixed]
mean_distances_10m_true = [np.mean(x) for x in distances_compared_to_10m_true]


DataFrame({'Path lengths down mixed': path_lengths_mixed,
           'mean distances to 5 m path': mean_distances_5m,
           'relative costs': [x['total cost'][0]* 10 for x in mixed],
           'mean distances to original resolution, false': mean_distances_10m_false,
           'mean distances to original resolution, true': mean_distances_10m_true},
          index=r_false)

,Path lengths down mixed,mean distances to 5 m path,relative costs,"mean distances to original resolution, false","mean distances to original resolution, true"
1,75948.436418,119.601576,96871.42,285.501047,47.214291
2,76265.389877,97.117364,94768.30,263.508125,74.191672
4,75960.953369,40.130071,92935.42,206.375485,100.160364
8,76062.374725,41.728524,91638.05,169.026848,137.341002
16,75979.532013,56.690747,90836.73,153.313887,152.711868
32,76332.080353,56.693706,90379.62,145.559320,162.089892
64,75448.892761,163.479456,90111.35,10.612876,272.363191
128,75448.892761,163.820110,89968.41,10.294056,272.706149


# costs

In [5]:
least_cost_path_5_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_false.json")
least_cost_path_5_false = least_cost_path_5_false.to_crs(25832)

lcp_false = [least_cost_path_5_false,
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_false.gpkg"),
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_false.gpkg"),
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_false.gpkg"),
               read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_false.gpkg")]
lcp_true = [read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_10_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_25_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_50_al_true.gpkg"),
              read_file("../results/least_cost_paths/least_cost_path_test_points_res_100_al_true.gpkg")]

resolutions = [5, 10, 25, 50, 100]
total_costs_false = np.array([x['total cost'][0] for x in lcp_false])
total_costs_true = np.array([x['total cost'][0] for x in lcp_true])
delta_total_costs = total_costs_false - total_costs_true

scaled_total_costs_false = np.array([x * y for x, y in zip(total_costs_false, resolutions)])
scaled_total_costs_true = np.array([x * y for x, y in zip(total_costs_true, resolutions)])
delta_scaled_costs = scaled_total_costs_false - scaled_total_costs_true

lengths_false = np.array([x.geometry.length[0] for x in lcp_false])
lengths_true = np.array([x.geometry.length[0] for x in lcp_true])
delta_scaled_costs_per_length = delta_scaled_costs / np.array(
    [x / 2 + y / 2 for x, y in zip(lengths_false, lengths_true)])

DataFrame({"total costs false": total_costs_false,
           "total costs true": total_costs_true,
           "scaled total costs false": scaled_total_costs_false,
           "scaled total costs true": scaled_total_costs_true,
           },
          index=resolutions)

,total costs false,total costs true,scaled total costs false,scaled total costs true
5,18665.923,19516.754,93329.615,97583.77
10,8931.245,9731.175,89312.450,97311.75
25,3354.869,3872.656,83871.725,96816.40
50,1409.023,2300.073,70451.150,115003.65
100,640.516,1572.268,64051.600,157226.80


# zonal stats
## 5 m

In [6]:
def compute_zonal_stat_path(base_raster: DataArray, path: GeoSeries, buffer: float, _cmap):
    stats = zonal_stats(path.buffer(buffer), base_raster.data[0],
                        affine=base_raster.rio.transform(), nodata=base_raster.rio.nodata,
                        categorical=True, category_map=_cmap)[0]

    stat_sum = np.sum(list(stats.values()))
    rel_stats = {k: (v / stat_sum * 100.0) for k, v in stats.items()}
    return rel_stats

In [7]:
def show_zonal_data(f_100, f_5, t_100, t_5) -> DataFrame:
    return DataFrame({'rel. statistic false buffer 100 m': f_100.values(),
           'rel. statistic false buffer 5 m': f_5.values(),
           'rel. statistic true buffer 100 m': t_100.values(),
           'rel. statistic true buffer 5 m': t_5.values(),
           },
          index=f_100.keys())

In [8]:
buffer_dist_100 = 100
buffer_dist_5 = 5

cmap = {0.1: 'Preferential', 0.5: 'No Restriction', 5.0: 'Restricted', 10.0: 'Strongly Restricted', 500: 'Prohibited'}

least_cost_path_5_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_false.gpkg")
least_cost_path_5_false = least_cost_path_5_false.to_crs(25832)
least_cost_path_5_true = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_5_al_true.gpkg")


raster_5f = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_False.tif'))
raster_5t = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_True.tif'))

rel_stats_5f_100 = compute_zonal_stat_path(raster_5f, least_cost_path_5_false, buffer_dist_100, cmap)
rel_stats_5f_5 = compute_zonal_stat_path(raster_5f, least_cost_path_5_false, buffer_dist_5, cmap)

rel_stats_5t_100 = compute_zonal_stat_path(raster_5t, least_cost_path_5_true, buffer_dist_100, cmap)
rel_stats_5t_5 = compute_zonal_stat_path(raster_5t, least_cost_path_5_true, buffer_dist_5, cmap)

show_zonal_data(rel_stats_5f_100, rel_stats_5f_5, rel_stats_5t_100, rel_stats_5t_5)

,rel. statistic false buffer 100 m,rel. statistic false buffer 5 m,rel. statistic true buffer 100 m,rel. statistic true buffer 5 m
Preferential,19.036771,30.062925,18.896877,28.469761
No Restriction,67.797856,65.320406,67.309240,66.440784
Restricted,0.908588,1.088352,1.291177,1.618986
Strongly Restricted,0.733488,0.667780,0.986639,0.496186
Prohibited,11.523298,2.860537,11.516067,2.974283


## 10 m

In [9]:
raster_10_f = open_rasterio(Path(r'./../results/weights/result_res_10_all_touched_False.tif')).rio.reproject_match(
    raster_5f)
raster_10_t = open_rasterio(Path(r'./../results/weights/result_res_10_all_touched_True.tif')).rio.reproject_match(
    raster_5t)

least_cost_path_10_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_10_al_false.gpkg")
least_cost_path_10_true = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_10_al_true.gpkg")


rel_stats_10f_100 = compute_zonal_stat_path(raster_10_f, least_cost_path_10_false, buffer_dist_100, cmap)
rel_stats_10f_5 = compute_zonal_stat_path(raster_10_f, least_cost_path_10_false, buffer_dist_5, cmap)
rel_stats_10t_100 = compute_zonal_stat_path(raster_10_t, least_cost_path_10_true, buffer_dist_100, cmap)
rel_stats_10t_5 = compute_zonal_stat_path(raster_10_t, least_cost_path_10_true, buffer_dist_5, cmap)

show_zonal_data(rel_stats_10f_100, rel_stats_10f_5, rel_stats_10t_100, rel_stats_10t_5)

,rel. statistic false buffer 100 m,rel. statistic false buffer 5 m,rel. statistic true buffer 100 m,rel. statistic true buffer 5 m
Preferential,19.595563,33.541397,18.921619,33.666907
No Restriction,68.533269,64.479301,66.608802,63.399944
Restricted,0.977264,0.730918,1.627109,1.377168
Strongly Restricted,0.799352,0.316947,1.374331,0.580356
Prohibited,10.094552,0.931436,11.468139,0.975625


## 25 m

In [10]:
raster_25f = open_rasterio(Path(r'./../results/weights/result_res_25_all_touched_False.tif')).rio.reproject_match(
    raster_5f)
raster_25t = open_rasterio(Path(r'./../results/weights/result_res_25_all_touched_True.tif')).rio.reproject_match(
    raster_5t)

least_cost_path_25_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_25_al_false.gpkg")
least_cost_path_25_true = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_25_al_true.gpkg")

rel_stats_25f_100 = compute_zonal_stat_path(raster_25f, least_cost_path_25_false, buffer_dist_100, cmap)
rel_stats_25f_5 = compute_zonal_stat_path(raster_25f, least_cost_path_25_false, buffer_dist_5, cmap)
rel_stats_25t_100 = compute_zonal_stat_path(raster_25t, least_cost_path_25_true, buffer_dist_100, cmap)
rel_stats_25t_5 = compute_zonal_stat_path(raster_25t, least_cost_path_25_true, buffer_dist_5, cmap)

show_zonal_data(rel_stats_25f_100, rel_stats_25f_5, rel_stats_25t_100, rel_stats_25t_5)

,rel. statistic false buffer 100 m,rel. statistic false buffer 5 m,rel. statistic true buffer 100 m,rel. statistic true buffer 5 m
Preferential,19.826572,34.227496,18.677156,31.925549
No Restriction,68.874543,64.912694,65.459597,65.514843
Restricted,0.956427,0.214952,1.955096,1.282586
Strongly Restricted,0.666827,0.091281,2.541035,0.703892
Prohibited,9.675631,0.553576,11.367116,0.573130


## 50 m

In [11]:
raster_50f = open_rasterio(Path(r'./../results/weights/result_res_50_all_touched_False.tif')).rio.reproject_match(
    raster_5f)
raster_50t = open_rasterio(Path(r'./../results/weights/result_res_50_all_touched_True.tif')).rio.reproject_match(
    raster_5t)

least_cost_path_50_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_50_al_false.gpkg")
least_cost_path_50_true = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_50_al_true.gpkg")

rel_stats_50f_100 = compute_zonal_stat_path(raster_50f, least_cost_path_50_false, buffer_dist_100, cmap)
rel_stats_50f_5 = compute_zonal_stat_path(raster_50f, least_cost_path_50_false, buffer_dist_5, cmap)
rel_stats_50t_100 = compute_zonal_stat_path(raster_50t, least_cost_path_50_true, buffer_dist_100, cmap)
rel_stats_50t_5 = compute_zonal_stat_path(raster_50t, least_cost_path_50_true, buffer_dist_5, cmap)

show_zonal_data(rel_stats_50f_100, rel_stats_50f_5, rel_stats_50t_100, rel_stats_50t_5)

,rel. statistic false buffer 100 m,rel. statistic false buffer 5 m,rel. statistic true buffer 100 m,rel. statistic true buffer 5 m
Preferential,20.424132,33.180815,9.144355,13.028230
No Restriction,67.975541,66.198445,75.682387,82.986624
Restricted,0.869934,0.076793,3.898266,1.956469
Strongly Restricted,0.670495,0.041596,4.172340,1.591995
Prohibited,10.059897,0.502352,7.102651,0.436681


## 100 m

In [12]:
raster_100f = open_rasterio('./../results/weights/result_res_100_all_touched_False.tif').rio.reproject_match(raster_5f)
raster_100t = open_rasterio(r'./../results/weights/result_res_100_all_touched_True.tif').rio.reproject_match(raster_5t)

least_cost_path_100_false = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_100_al_false.gpkg")
least_cost_path_100_true = read_file("./../results/least_cost_paths/least_cost_path_test_points_res_100_al_true.gpkg")

rel_stats_100f_100 = compute_zonal_stat_path(raster_100f, least_cost_path_100_false, buffer_dist_100, cmap)
rel_stats_100f_5 = compute_zonal_stat_path(raster_100f, least_cost_path_100_false, buffer_dist_5, cmap)
rel_stats_100t_100 = compute_zonal_stat_path(raster_100t, least_cost_path_100_true, buffer_dist_100, cmap)
rel_stats_100t_5 = compute_zonal_stat_path(raster_100t, least_cost_path_100_true, buffer_dist_5, cmap)

show_zonal_data(rel_stats_100f_100, rel_stats_100f_5, rel_stats_100t_100, rel_stats_100t_5)

,rel. statistic false buffer 100 m,rel. statistic false buffer 5 m,rel. statistic true buffer 100 m,rel. statistic true buffer 5 m
Preferential,21.116438,30.733593,6.972863,10.132879
No Restriction,69.125830,68.812704,73.807723,81.943444
Restricted,1.105329,0.038341,5.540127,3.911108
Strongly Restricted,0.720938,0.019171,8.512545,3.610002
Prohibited,7.931465,0.396191,5.166743,0.402566


# lines -> buffered Polygons

In [13]:
def create_buffer_polygon(line1: LineString, line2: LineString, crs, buffer) -> GeoDataFrame:
    x = list(line1.xy[0])
    y = list(line1.xy[1])

    x.extend(list(reversed(line2.xy[0])))
    y.extend(list(reversed(line2.xy[1])))

    poly = make_valid(Polygon(zip(x, y))).buffer(buffer)

    return GeoDataFrame({'buffer': [buffer, ], 'geometry': [poly, ]}, crs=crs)

In [18]:
[create_buffer_polygon(f.geometry.values[0], t.geometry.values[0], 25832, 2 * max_dist).to_file(s) for f, t, max_dist, s in
 zip(paths_false, paths_true, max_distances,
     [f"./../results/Buffer_Polygons/least_cost_path_res_{x}_buffer.gpkg" for x in resolutions])]

[None, None, None, None, None]

# Mask the 2.5 m and 5 m resolution Raster by the Buffer (from the 50 m resolution Paths)
Masking the original 50 m resolution cost raster by the polygon, created from the buffered 50 m resolution the least cost path.
The buffer is two times the maximum minimum distance between the paths.
## masking the 5 m resolution raster

In [19]:
def create_statistic(raster: list, names: list) -> DataFrame:
    raster_y = np.array([x.shape[1] for x in raster])
    raster_x = np.array([x.shape[2] for x in raster])
    counts = raster_y * raster_x
    nodata_counts = np.array([np.sum(x.data == x.rio.nodata) for x in raster])

    return DataFrame(data={'height': raster_y,
                           'width': raster_x,
                           'pixel_count': counts,
                           'pixel_percentage': counts / np.max(counts) * 100.0,
                           'nodata_count': nodata_counts,
                           'nodata_percentage': nodata_counts / counts * 100.0,
                           'kept_percentage': (counts - nodata_counts) / np.max(counts) * 100.0},

                     index=names)

In [20]:
mask_50 = read_file("./../results/Buffer_Polygons/least_cost_path_res_50_buffer.gpkg")

raster_5f = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_False.tif'))
raster_5t = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_True.tif'))

raster_5f_clipped = raster_5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_5f_clipped, Path(r'./../results/weights/Clipped/result_res_5_all_touched_False_clipped.tif'))
raster_5f_clipped_dropped = raster_5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_5f_clipped_dropped,
                 Path(r'./../results/weights/Clipped/result_res_5_all_touched_False_clipped_drop.tif'))

raster_5t_clipped = raster_5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_5t_clipped, Path(r'./../results/weights/Clipped/result_res_5_all_touched_True_clipped.tif'))
raster_5t_clipped_dropped = raster_5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_5t_clipped_dropped,
                 Path(r'./../results/weights/Clipped/result_res_5_all_touched_True_clipped_drop.tif'))

raster_5 = [raster_5f, raster_5f_clipped, raster_5f_clipped_dropped, raster_5t, raster_5t_clipped,
            raster_5t_clipped_dropped]
raster_5_names = ["raster_5f", "raster_5f_clipped", "raster_5f_clipped_dropped", "raster_5t", "raster_5t_clipped",
                  "raster_5t_clipped_dropped"]
create_statistic(raster_5, raster_5_names)

,height,width,pixel_count,pixel_percentage,nodata_count,nodata_percentage,kept_percentage
raster_5f,17481,11270,197010870,100.000000,75314390,38.228545,61.771455
raster_5f_clipped,17481,11270,197010870,100.000000,160168500,81.299321,18.700679
raster_5f_clipped_dropped,11871,8984,106649064,54.133594,69806694,65.454577,18.700679
raster_5t,17481,11270,197010870,100.000000,75274817,38.208459,61.791541
raster_5t_clipped,17481,11270,197010870,100.000000,160158784,81.294389,18.705611
raster_5t_clipped_dropped,11871,8984,106649064,54.133594,69796978,65.445467,18.705611


## masking the 2.5 m resolution raster

In [21]:
mask_50 = read_file("./../results/Buffer_Polygons/least_cost_path_res_50_buffer.gpkg")

raster_2p5f = open_rasterio(Path(r'./../results/weights/result_res_2.5_all_touched_False.tif'))
raster_2p5t = open_rasterio(Path(r'./../results/weights/result_res_2.5_all_touched_True.tif'))

raster_2p5f_clipped = raster_2p5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_2p5f_clipped,
                 Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_False_clipped.tif'))
raster_2p5f_clipped_dropped = raster_2p5f.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_2p5f_clipped_dropped,
                 Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_False_clipped_drop.tif'))

raster_2p5t_clipped = raster_2p5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=False)
write_compressed(raster_2p5t_clipped, Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_True_clipped.tif'))
raster_2p5t_clipped_dropped = raster_2p5t.rio.clip(mask_50.geometry.values, mask_50.crs, drop=True)
write_compressed(raster_2p5t_clipped_dropped,
                 Path(r'./../results/weights/Clipped/result_res_2p5_all_touched_True_clipped_drop.tif'))

raster_2p5 = [raster_2p5f, raster_2p5f_clipped, raster_2p5f_clipped_dropped, raster_2p5t, raster_2p5t_clipped,
              raster_2p5t_clipped_dropped]
raster_2p5_names = ['raster_2p5f', 'raster_2p5f_clipped', 'raster_2p5f_clipped_dropped', 'raster_2p5t',
                    'raster_2p5t_clipped', 'raster_2p5t_clipped_dropped']
create_statistic(raster_2p5, raster_2p5_names)

,height,width,pixel_count,pixel_percentage,nodata_count,nodata_percentage,kept_percentage
raster_2p5f,34962,22540,788043480,100.000000,301257639,38.228555,61.771445
raster_2p5f_clipped,34962,22540,788043480,100.000000,640673252,81.299226,18.700774
raster_2p5f_clipped_dropped,23741,17969,426602029,54.134326,279231801,65.454869,18.700774
raster_2p5t,34962,22540,788043480,100.000000,301178497,38.218513,61.781487
raster_2p5t_clipped,34962,22540,788043480,100.000000,640653579,81.296730,18.703270
raster_2p5t_clipped_dropped,23741,17969,426602029,54.134326,279212128,65.450258,18.703270


# Mask the 2.5 and 5 m resolution raster by the non-reachable pixels

In [22]:
def mask_by_aggregated_nodata(original: DataArray, aggregate: DataArray):
    aggregate_resized = aggregate.copy().rio.reproject_match(raster_5f)

    original_masked = original.copy()
    original_masked.data[aggregate_resized == aggregate_resized.rio.nodata] = original.rio.nodata
    return original_masked

In [23]:
agg_cost_50_raster = open_rasterio("./../results/weights/aggregated/result_res_50_all_touched_False_agg.tif")
limit = 2149
agg_cost_50_raster.data[agg_cost_50_raster > limit] = agg_cost_50_raster.rio.nodata

raster_5f = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_False.tif'))
raster_5t = open_rasterio(Path(r'./../results/weights/result_res_5_all_touched_True.tif'))


# mask cost raster by nodata values in aggregated cost raster
raster_5f_masked = mask_by_aggregated_nodata(raster_5f, agg_cost_50_raster)
write_compressed(raster_5f_masked,
                 Path("./../results/weights/Clipped/result_res_5_all_touched_False_mask_by_aggregated_costs.tif"))

raster_5t_masked = mask_by_aggregated_nodata(raster_5t, agg_cost_50_raster)
write_compressed(raster_5t_masked,
                 Path("./../results/weights/Clipped/result_res_5_all_touched_True_mask_by_aggregated_costs.tif"))

raster_agg = [raster_5f, raster_5f_masked, raster_5t, raster_5t_masked]
raster_agg_names = ['raster_5f', 'raster_5f_masked', 'raster_5t', 'raster_5t_masked']
create_statistic(raster_agg, raster_agg_names)

,height,width,pixel_count,pixel_percentage,nodata_count,nodata_percentage,kept_percentage
raster_5f,17481,11270,197010870,100.0,75314390,38.228545,61.771455
raster_5f_masked,17481,11270,197010870,100.0,93820901,47.622195,52.377805
raster_5t,17481,11270,197010870,100.0,75274817,38.208459,61.791541
raster_5t_masked,17481,11270,197010870,100.0,93813930,47.618657,52.381343


# Distances all touched False

In [24]:
distances_false = np.array([Point(p).distance(least_cost_path_100_false.geometry.values[0]) for p in
                            least_cost_path_5_false.geometry.values[0].coords])
mean_min_distances_false = np.mean(distances_false)
mean_min_distances_false

257.9733343117007

# Distances all touched True


In [25]:
distances_true = np.array([Point(p).distance(least_cost_path_100_true.geometry.values[0]) for p in
                           least_cost_path_5_true.geometry.values[0].coords])
mean_min_distances_true = np.mean(distances_true)
mean_min_distances_true

2109.436322189748